In [3]:
%matplotlib inline
%run ../../path_import.py
from plots import *
from training_reparam import *
from eubo import *
from model_reparam import *

import probtorch
print('probtorch:', probtorch.__version__, 
      'torch:', torch.__version__, 
      'cuda:', torch.cuda.is_available())

probtorch: 0.0+5a2c637 torch: 1.1.0 cuda: True


In [4]:
data_path = "../gmm_dataset_c5k"
Data = torch.from_numpy(np.load(data_path + '/obs.npy')).float()
_, N, D = Data.shape
K = 3 ## number of clusters
## Model Parameters
MCMC_SIZE = 20
SAMPLE_SIZE = 10
NUM_HIDDEN_LOCAL = 32
## Training Parameters
BATCH_SIZE = 20
NUM_EPOCHS = 1000
LEARNING_RATE = 1e-4
CUDA = torch.cuda.is_available()
PATH = 'ag-pr-%dsteps-%dsamples' % (MCMC_SIZE, SAMPLE_SIZE)

DEVICE = torch.device('cuda:0')

In [5]:
# initialization
enc_z = Gibbs_z(K, CUDA, DEVICE)
enc_eta = Enc_eta(K, D, CUDA, DEVICE)
if CUDA:
    enc_eta.cuda().to(DEVICE)
optimizer =  torch.optim.Adam(list(enc_eta.parameters()),lr=LEARNING_RATE, betas=(0.9, 0.99))

In [6]:
RESAMPLE = True
DETACH=True
train_cfz_reparam(Eubo_cfz_pr_eta, enc_eta, enc_z, optimizer, Data, K, NUM_EPOCHS, MCMC_SIZE, SAMPLE_SIZE, BATCH_SIZE, PATH, CUDA, DEVICE, RESAMPLE=RESAMPLE, DETACH=DETACH)

KeyboardInterrupt: 

In [ ]:
torch.save(enc_eta.state_dict(), '../weights/enc-eta-%s' + PATH)

In [ ]:
BATCH_SIZE_TEST = 50
RESAMPLE= False
obs, q_eta, q_z, symkl_test, eubo_test, elbo_test = test(Eubo_cfz_pr_eta, enc_eta, enc_z, Data, K, MCMC_SIZE, SAMPLE_SIZE, BATCH_SIZE_TEST, CUDA, DEVICE, RESAMPLE=RESAMPLE)

In [ ]:
%time plot_samples(obs, q_eta, q_z, PATH)

In [ ]:
incremental_gap = symkl_test.cpu().data.numpy()[1:]
M = incremental_gap.shape[0]
overall_gap = np.zeros(M)
for m in range(M):
    overall_gap[m] = incremental_gap[:m+1].sum()

In [ ]:
fig = plt.figure(figsize=(12,6))
ax = fig.add_subplot(111)
plt.yscale("log")
ax.plot(incremental_gap, label="incremental gap")
ax.plot(overall_gap, label='overall gap')
ax.legend(fontsize=14)
ax.set_xlabel('Steps')

In [ ]:
incremental_gap